In [ ]:
import os, sys

# if os.path.exists('analysis'):
#     os.system('rm -rf analysis')

# !git clone https://github.com/element-fi/analysis.git

# sys.path.insert(1, os.getcwd())

In [ ]:
import json, numbers, math, time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sim import YieldSimulator#, ElementPricingModel, Market, YieldSpacev2PricingModel 

pd.set_option("float_format",'{:,.6f}'.format)

In [ ]:
random_seed = 3
config = {
    'min_fee': 0.,
    'max_fee': 0.5,
    'floor_fee': 0,
    't_min': 0.001,
    't_max': 1.0,
    'base_asset_price': 2500., # aka market price
    'min_target_liquidity': 100000.,
    'max_target_liquidity': 10000000.,
    'min_target_volume': 2e5,
    'max_target_volume': 6e5,
    'min_pool_apy': 0.5,
    'max_pool_apy': 50,
    'min_vault_age': 0.,
    'max_vault_age': 2,
    'min_vault_apy': 0.,
    'max_vault_apy': 10.,
    'precision': 8,
    'pricing_model_name': 'YieldSpace',
    'tokens': ['base', 'fyt'],
    'trade_direction': 'out',
    'days_until_maturity': 90,
    'num_trading_days': 90, # should be <= days_until_maturity
    'rng': np.random.default_rng(random_seed),
}

In [ ]:
start_time = time.time()
num_runs = 0

simulator = YieldSimulator(**config)
simulator.set_random_variables()

apr = 10
override_dict = {
    'pricing_model_name': 'Element',
    'vault_apy': [apr,]*config['num_trading_days'],
    'init_pool_apy' : apr,
    'init_vault_age': 1,
    'target_daily_volume': 5*1e5,
    'target_liquidity': 10*1e6,
    'fee_percent' : 0.1,
}
#simulator.reset_rng(np.random.default_rng(random_seed)) # do this to make sure simulations run over the same trade sequence
simulator.run_simulation(override_dict)
num_runs += 1

floor_fee_list = [0, 5]
override_dict['pricing_model_name'] = 'YieldSpacev2'
for floor_fee in floor_fee_list:
    override_dict['floor_fee'] = floor_fee
    #simulator.reset_rng(np.random.default_rng(random_seed)) # do this to make sure simulations run over the same trade sequence
    simulator.run_simulation(override_dict)
    num_runs += 1

end_time = time.time()
print(f'Total time for {num_runs} runs was {end_time-start_time:.3f} seconds; which is {(end_time-start_time)/num_runs:.3f} seconds per run')

In [ ]:
def format_trades(analysis_dict):
    trades = pd.DataFrame.from_dict(analysis_dict)    # construct simulation dataframe output
    # calculate derived variables across runs
    trades['time_diff'] = trades.time_until_end.diff()
    trades['time_diff_shift'] = trades.time_until_end.shift(-1).diff()
    trades.loc[len(trades) - 1, 'time_diff_shift'] = 1
    trades['fee_in_usd'] = trades.fee #* trades.base_asset_price
    trades['fee_in_bps'] = trades.fee / trades.out_without_fee * 100 * 100
    x_liquidity_usd = trades.x_reserves * trades.base_asset_price
    y_liquidity_usd = trades.y_reserves * trades.base_asset_price * trades.spot_price
    trades['total_liquidity_usd'] = x_liquidity_usd + y_liquidity_usd
    trades['trade_volume_usd'] = trades.out_with_fee #* trades.base_asset_price
    # pr is the percent change in spot price since day 1
    trades['pr'] = trades.loc[:, 'spot_price'] - trades.loc[0, 'spot_price'] # this is APR (does not include compounding)
    # pu takes that percent change and normalizes it to be equal to init_price_per_share at the beginning, so you can compare its progression vs. price_per_share
    trades['pu'] = (trades.pr + 1) * trades.init_price_per_share # this is APR (does not include compounding)

    # create explicit column that increments per trade
    #trades = trades.reset_index().rename(columns = {'index': 'trade_number'})
    trades = trades.reset_index()

    keep_columns = [
    'day',
    'model_name',
    ]
    df_fees_volume = trades.groupby(keep_columns).agg({
        'trade_volume_usd': ['sum'],
        'fee_in_usd': ['mean', 'std', 'min', 'max', 'sum'],
        'fee_in_bps': ['mean', 'sum'],
    })
    df_fees_volume.columns = ['_'.join(col).strip() for col in df_fees_volume.columns.values]
    df_fees_volume['fee_in_usd_sum_cum'] = 0
    df_fees_volume = df_fees_volume.reset_index()
    for model in df_fees_volume.model_name.unique():
        df_fees_volume.loc[df_fees_volume.model_name == model, 'fee_in_usd_sum_cum'] = df_fees_volume.loc[df_fees_volume.model_name == model, 'fee_in_usd_sum'].cumsum()
    return [trades,df_fees_volume]

[trades,df_fees_volume] = format_trades(simulator.analysis_dict)

In [ ]:
print(f'Simulator contains {trades.shape[1]} variables tracked over {trades.shape[0]} simulations.')
print(f'Trades dataframe variables:\n'+'\t'.join(trades.columns))

In [ ]:
print(f'\nTrades dataframe:')
display(pd.concat([trades.head(4), trades.tail(2)]).T)
print(f'Rows at maturity:')
display(trades.loc[(trades.time_diff>0) | (trades.time_diff_shift>0) | (trades.index == trades.index.max()), :])
trades.loc[trades.run_number == 0, 'token_in'].to_csv('token_in_simclass.csv')

In [ ]:
# create runs dataframe from the last trade in each run (using time_diff instead would pick the 1st)
changing_columns = [ # These columns should change with run aggregation
    'run_number',
    'model_name',
    'num_orders',
    'pool_apy',
]
const_columns = [ # These should not change with run aggregation
    't_stretch',
    'target_liquidity',
    'target_daily_volume',
    'fee_percent',
    'floor_fee',
    'init_vault_age',
    'vault_apy',
    'days_until_maturity',
    'num_trading_days'
] 
interesting_columns = changing_columns + const_columns
runs = trades.loc[trades.time_diff_shift > 0, :].loc[:, interesting_columns].copy()
# variables that change per trade represent the last value in the run, rename the useful ones
runs.rename(columns={'current_apy':'ending_apy', 'num_orders':'total_orders'}, inplace=True)
# there is one row per "run", set that as a named column
runs.set_index('run_number', inplace=True)
display(runs.T)

In [ ]:
print('Standard deviations of constant columns (should all be zero):')
display(trades.groupby('run_number')[[x for x in const_columns]].std().T) # check if these are really constant

In [ ]:
# target columns for taking averages & sums
mean_columns = [
    'time_until_end',
    'init_price_per_share',
    'base_asset_price',
    'spot_price',
    'out_without_fee_slippage',
    'x_reserves',
    'out_without_fee',
    'fee',
    'trade_amount',
    'out_with_fee',
    'day',
    'fee_in_bps',
    'y_reserves',
    'total_supply'
]
sum_columns = [
    'fee',
    'out_with_fee',
    'out_without_fee',
    'out_without_fee_slippage',
    'trade_amount'
]
trades_mean = trades.groupby(['run_number'])[mean_columns].mean()
trades_mean.columns = ['mean_'+col for col in trades_mean.columns]
trades_sum = trades.groupby(['run_number'])[sum_columns].sum()
trades_sum.columns = ['sum_'+col for col in trades_sum.columns]
display(pd.concat([runs, trades_mean, trades_sum], axis=1).T)

In [ ]:
display(trades.loc[:, ['day', 'num_orders', 'trade_amount', 'spot_price', 'pr', 'pu']])

In [ ]:
display(trades.groupby(['model_name', 'token_in']).agg({
    'trade_volume_usd': ['sum'],
    'run_trade_number': ['count'],
    'fee_in_bps': ['mean', 'std', 'min', 'max', 'sum'],
    'day': ['mean', 'min', 'max'],
    })
)

In [ ]:
df_to_display = pd.DataFrame()
for model in trades.model_name.unique(): # for each model (per run?)
    df_to_display = pd.concat([df_to_display, trades.loc[trades.model_name==model, :].head(1)])
df_to_display=df_to_display.set_index('model_name',drop=True)
diffs = []
ratios = []
for i in range(0, df_to_display.shape[1]):
    if isinstance(df_to_display.iloc[0, i], numbers.Number):
        diffs.append(df_to_display.iloc[1, i] - df_to_display.iloc[0, i])
        ratios.append(df_to_display.iloc[1, i] / df_to_display.iloc[0, i])
    else:
        diffs.append(df_to_display.iloc[0, i])
        ratios.append(df_to_display.iloc[0, i])
df_to_display.loc['diff'] = diffs
df_to_display.loc['ratio'] = ratios
display(df_to_display.loc[:, (df_to_display.iloc[0, :].values != df_to_display.iloc[1, :].values) | (df_to_display.columns.isin(['price_per_share', 'init_price_per_share']))].T)

In [ ]:
display(df_fees_volume)

In [ ]:
numPlots = 6

figsize = (24, 18)
fig, ax = plt.subplots(ncols=1, nrows=numPlots, gridspec_kw = {'wspace':0, 'hspace':0, 'height_ratios':np.ones(numPlots)})
fig.patch.set_facecolor('white')   # set fig background color to white

colors = list(plt.rcParams['axes.prop_cycle'].by_key()['color'])
model_colors = {model:colors[i] for i, model in enumerate(df_fees_volume.model_name.unique())}

# fees
current_plot = 0
for model in df_fees_volume.model_name.unique():
    ax[current_plot] = df_fees_volume.loc[df_fees_volume.model_name==model, :].plot(
        x='day', y='fee_in_usd_sum', figsize=figsize, ax=ax[current_plot], color=model_colors[model], label=model)
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Fees (USD)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].xaxis.set_ticklabels([])
title = (
    f'Data Sussy· Initial pool APY: {trades.pool_apy[0]:.2f}% Time Stretch: {trades.t_stretch[0]:.2f} Maturity: {trades.days_until_maturity[0]} days\n'
    +f'Target Liquidity: {trades.target_liquidity[0]:,.0f} Target Daily Volume: {trades.target_daily_volume[0]:,.0f} Percent Fees: {trades.fee_percent[0]:.2f}%'
)
ax[current_plot].set_title(title, fontsize=20)
ax[current_plot].legend(fontsize=18)

# fees cumulative
current_plot += 1
for model in df_fees_volume.model_name.unique():
    ax[current_plot] = df_fees_volume.loc[df_fees_volume.model_name==model, :].plot(
        x='day', y='fee_in_usd_sum_cum', figsize=figsize, ax=ax[current_plot], color=model_colors[model], label=f'{model}_cum {df_fees_volume.loc[df_fees_volume.model_name==model, :].fee_in_usd_sum_cum.iloc[-1]:,.0f}')
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Cumulative Fees (USD)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)

# pool APYs
current_plot += 1
for model in df_fees_volume.model_name.unique(): # for each model (per run?)
    ax[current_plot] = trades.loc[trades.model_name==model, :].plot(
        x='run_trade_number', # could also do num_orders if you want to know the apy for a given volume of trade
        y='pool_apy', figsize=figsize, ax=ax[current_plot], color=model_colors[model], label=f'{model}')
ax[current_plot].set_xlabel('')
# ax[current_plot].set_ylabel('Pool APY', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)

# vault APYs
# current_plot += 1
display(df_fees_volume.model_name.unique()[0])
for model in [df_fees_volume.model_name.unique()[0]]: # only show the first model, vault APYs are the same for all models
    ax[current_plot] = trades.loc[trades.model_name==model, :].plot(
        x='run_trade_number', y='vault_apy', figsize=figsize, ax=ax[current_plot], linestyle='-', color='black', label=f'vault')
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('APY', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)

# trade volume
current_plot += 1
for model in df_fees_volume.model_name.unique():
    ax[current_plot] = df_fees_volume.loc[df_fees_volume.model_name==model,:].plot(
    x='day', y='trade_volume_usd_sum', kind='line', ax=ax[current_plot], color=model_colors[model], label=model)
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Volume (USD)', fontsize=18)
ax[current_plot].tick_params(axis = 'both', labelsize=12)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].ticklabel_format(style='plain', axis='y')
ax[current_plot].legend(fontsize=18)

# reserves
current_plot += 1
for model in df_fees_volume.model_name.unique():
    ax[current_plot] = trades.loc[trades.model_name==model,:].plot(
        x='day', y='x_reserves', kind='line', ax=ax[current_plot], color=model_colors[model], linestyle='-', label=model+'_x')
    ax[current_plot] = trades.loc[trades.model_name==model,:].plot(
        x='day', y='y_reserves', kind='line', ax=ax[current_plot], color=model_colors[model], linestyle='--', label=model+'_y')
ax[current_plot-2].legend(fontsize=18)
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Liquidity (USD)', fontsize=18)
ax[current_plot].tick_params(axis = 'both', labelsize=12)
ax[current_plot].grid(visible=True,linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].ticklabel_format(style='plain', axis='y')
ax[current_plot].legend(fontsize=18)

# price per share
current_plot += 1
for model in trades.model_name.unique():
    ax[current_plot] = trades.loc[trades.model_name==model].plot(
        x='day', y='price_per_share', figsize=figsize, ax=ax[current_plot], kind='line', color=model_colors[model],
        label=(
            f'{model} growth={trades.loc[trades.model_name==model].loc[:, "price_per_share"].values[-1] / trades.loc[trades.model_name==model].loc[:, "price_per_share"].values[0] - 1:.3%}')
    )
ax[current_plot].set_ylabel('Price per share', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
# ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)

# spot price
# current_plot += 1
for model in trades.model_name.unique():
    ax[current_plot] = trades.loc[trades.model_name==model].plot(
        x='day', y='pu', figsize=figsize, ax=ax[current_plot], kind='line', color=model_colors[model],
        label=(
            f'{model} growth={trades.loc[trades.model_name==model].loc[:, "pu"].values[-1] / trades.loc[trades.model_name==model].loc[:, "pu"].values[0]-1:.3%}')
    )
ax[current_plot].set_ylabel('Spot price', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
# ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)
ax[current_plot].set_xlabel('Day', fontsize=18)

plt.show()